In [35]:
import yaml
import torch
from ultralytics.nn.tasks import parse_model, DetectionModel

# --- 1) 설정할 값들 ----------------------------------------
yaml_path = '12/yolo12.yaml'     # 원본 모델 YAML
scale_key = 'n'                  # 사용할 스케일 키 (n, s, m, l, x)
new_depth = 0.50                 # depth_multiple 또는 scales[scale_key][0]
new_width = 0.25                 # width_multiple 또는 scales[scale_key][1]
# ---------------------------------------------------------

# 2) YAML 불러오기
with open(yaml_path, 'r', encoding='utf-8') as f:
    d = yaml.safe_load(f)

# 3) depth/width 수정
if 'scales' in d and scale_key in d['scales']:
    d['scales'][scale_key][0] = new_depth
    d['scales'][scale_key][1] = new_width
    d['scale'] = scale_key  # 실제 사용할 스케일 지정
else:
    d['depth_multiple'] = new_depth
    d['width_multiple'] = new_width

# 4) 모델 파싱 & 파라미터 수 세기
try:
    model_seq, _ = parse_model(d, ch=3, verbose=False)
except Exception:
    model_obj = DetectionModel(d, ch=3, nc=d.get('nc', 1), verbose=False)
    model_seq = model_obj.model

n_params = sum(p.numel() for p in model_seq.parameters() if p.requires_grad)
print(f"▶ 수정된 모델 파라미터 수: {n_params/1e6:.2f}M")

# 5) 수정된 YAML 파일로 저장
out_yaml = '12/yolo12_n_custom.yaml'
with open(out_yaml, 'w', encoding='utf-8') as f:
    yaml.dump(d, f, sort_keys=False)

print(f"▶ 수정된 모델 정의를 '{out_yaml}' 에 저장했습니다.")

▶ 수정된 모델 파라미터 수: 3.71M
▶ 수정된 모델 정의를 '12/yolo12_n_custom.yaml' 에 저장했습니다.
